In [1]:
%run -i /opt/setup_spark.py

Create a local file just on the Driver, this is not accessible by the Spark Cluster

In [2]:
!echo "3 Downtown 400000\n2 Downtown 240000\n3 Hilltop 650000" > houses.txt

In [3]:
!cat houses.txt

3 Downtown 400000
2 Downtown 240000
3 Hilltop 650000


Copy the file to the distributed file system HDFS

In [4]:
!hdfs dfs -rm houses.txt

2015-11-16 23:12:40,751 INFO  [main] fs.TrashPolicyDefault (TrashPolicyDefault.java:initialize(92)) - Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted houses.txt


In [5]:
!hdfs dfs -copyFromLocal houses.txt

Verify that the file is replicated 3 times on all nodes `10.128.1.6`, `10.128.1.7`, `10.128.1.8`

In [6]:
!hdfs fsck /user/root/houses.txt -files -blocks -locations -racks  

Connecting to namenode via http://sparkcluster-2:50070
FSCK started by root (auth:SIMPLE) from /10.128.1.5 for path /user/root/houses.txt at Mon Nov 16 23:12:46 UTC 2015
/user/root/houses.txt 53 bytes, 1 block(s):  OK
0. BP-545221491-10.128.1.5-1447628910426:blk_1073741830_1006 len=53 repl=3 [/default-rack/10.128.1.6:50010, /default-rack/10.128.1.7:50010, /default-rack/10.128.1.8:50010]

Status: HEALTHY
 Total size:	53 B
 Total dirs:	0
 Total files:	1
 Total symlinks:		0
 Total blocks (validated):	1 (avg. block size 53 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	3.0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)
 Number of data-nodes:		3
 Number of racks:		1
FSCK ended at Mon Nov 16 23:12:46 UTC 2015 in 2 milliseconds


The filesystem under path '/user/root/houses.txt' is HEALTHY


In [7]:
text_RDD = sc.textFile("houses.txt")

In [8]:
text_RDD.collect()

[u'3 Downtown 400000', u'2 Downtown 240000', u'3 Hilltop 650000']

In [17]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]))

In [18]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [19]:
house_prices_RDD.collect()

[(u'Downtown', 400000.0), (u'Downtown', 240000.0), (u'Hilltop', 650000.0)]

In [20]:
def reducer_sum(a,b):
    return a+b

In [21]:
house_prices_RDD.reduceByKey(reducer_sum).collect()

[(u'Downtown', 640000.0), (u'Hilltop', 650000.0)]

## filter

In [15]:
def is_in_downtown(record):
    return record[0] == "Downtown"
house_prices_downtown_RDD = house_prices_RDD.filter(is_in_downtown)

In [16]:
house_prices_downtown_RDD.collect()

[(u'Downtown', 400000), (u'Downtown', 240000)]